# CSCC11 - Introduction to Machine Learning, Fall 2022, Assignment 1

In [55]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import statistics
from scipy import stats

In [56]:
#TO-DO
"""
Read the csv file into a DataFrame - df
"""
df = pd.read_csv('Admission_Predict.csv')

In [57]:
"""
Print the DataFrame
"""
print(df)

     Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  \
0             1        337          118                  4  4.5   4.5  9.65   
1             2        324          107                  4  4.0   4.5  8.87   
2             3        316          104                  3  3.0   3.5  8.00   
3             4        322          110                  3  3.5   2.5  8.67   
4             5        314          103                  2  2.0   3.0  8.21   
..          ...        ...          ...                ...  ...   ...   ...   
395         396        324          110                  3  3.5   3.5  9.04   
396         397        325          107                  3  3.0   3.5  9.11   
397         398        330          116                  4  5.0   4.5  9.45   
398         399        312          103                  3  3.5   4.0  8.78   
399         400        333          117                  4  5.0   4.0  9.66   

     Research  Chance of Admit   
0           1    

In [58]:
#TO-DO
"""
Print the length of the DataFrame.
Print the column names of the DataFrame.
"""
print(len(df))
for col in df.columns:
    print(col)

400
Serial No.
GRE Score
TOEFL Score
University Rating
SOP
LOR 
CGPA
Research
Chance of Admit 


In [59]:
#TO-DO
"""
Define an “X” array that would hold our independent features for regression purposes.  
Define a "Y" array that would hold our target variable.

Print the shape of both the arrays.
"""
x = np.delete(np.delete(df.to_numpy(),8,1),0,1)
print(x.shape)
y = df[["Chance of Admit "]].to_numpy()
print(y.shape)

(400, 7)
(400, 1)


## Split the data 

In [60]:
#TO-DO
"""
Split the dataset into train dataset and test dataset.
Set the random state to any number in order to maintain consistency while generating random numbers over several runs.
"""
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=10)

# Linear Regression

In [61]:
#TO-DO
def find_optimal_parameters(x, y):
    """ Compute closed form solution for linear regression!
    Optimal weight w* in linear regression is given by w* = (X^T X)^(-1) X^T Y
    
    Args:
    - x (ndarray (Shape: (N, D))): A NxD matrix corresponding to the inputs.
    - y (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    
    Output:
    - w (ndarray (Shape: (D+1, 1))): A D-column vector corresponding to the bias and weights of the linear model.
    """
    # Pad 1's for the bias term, Why?
    x = np.hstack((np.ones((x.shape[0], 1)), x))

    # Note that we could use pseudoinverse here instead: np.linalg.pinv
    # @ is alias for matmul
    p1 = np.linalg.pinv(np.matmul(x.transpose(),x))
    p2 = np.matmul(x.transpose(),y)
    w = np.matmul(p1,p2)
    return w

### Train linear regression model using training data

In [62]:
#TO-DO
def get_pred_Y(trained_w, X_pred):
    """ Return predicted Y
    Args:
    - trained_w (ndarray (Shape: (D+1, 1))): A (D+1)x1 column vector containing linear regression weights.
    - X_pred (ndarray (Shape: (N, D))): A NxD matrix corresponding to the prediction inputs.
    
    Output:
    - pred_Y (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the predicted outputs.
    """
    pad_Y     = np.hstack((np.ones((X_pred.shape[0], 1)), X_pred))
    pred_Y    = np.matmul(pad_Y,trained_w)
    return pred_Y

In [63]:
#TO-DO
def get_mae(Y_truth, Y_pred):
    """ Return Mean absolute error
    Args:
    - Y_truth (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the actual outputs.
    - Y_pred (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the predicted outputs.
    
    Output:
    - MSE (ndarray (Shape: (1,))).
    """
    'check if both inputs are of the same shape'
    print(Y_truth.shape == Y_pred.shape)
    mae = mean_absolute_error(Y_truth, Y_pred)
    return mae

def get_mse(Y_truth, Y_pred):
    """ Return Mean squared error
    Args:
    - Y_truth (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the actual outputs.
    - Y_pred (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the predicted outputs.
    
    Output:
    - MSE (ndarray (Shape: (1,))).
    """
    'check if both inputs are of the same shape'
    print(Y_truth.shape == Y_pred.shape)
    mse    = mean_squared_error(Y_truth, Y_pred)
    return mse

### Get predictions on train data

In [64]:
w_optimal = find_optimal_parameters(X_train, Y_train)
print(w_optimal)

[[-1.36241905]
 [ 0.00223841]
 [ 0.00214531]
 [ 0.00881854]
 [-0.00313771]
 [ 0.02090126]
 [ 0.12282707]
 [ 0.01047986]]


In [65]:
pred_Y    = get_pred_Y(w_optimal, X_train)
print('train error (MSE): ', get_mse(Y_train, pred_Y))
print('train error (MAE): ', get_mae(Y_train, pred_Y))

True
train error (MSE):  0.0036037699293513708
True
train error (MAE):  0.043405490235927255


### Get predictions and performance on test data

In [33]:
pred_Y    = get_pred_Y(w_optimal, X_test)
print('test error (MSE):: ', get_mse(Y_test, pred_Y))
print('test error (MAE): ', get_mae(Y_test, pred_Y))

True
test error (MSE)::  0.005058117226211304
True
test error (MAE):  0.04831836660743023


# Silouette Coefficient

In [67]:
#TO-DO
n_silhouette = [2,3,4,5,6,7,8,9,10]
silhouetter_coefficients = []
kmeans_kwargs= {
    "init":"k-means++",
    "n_init":30,
    "max_iter":250,
    "random_state":2
}

for i in n_silhouette:
    clusterer = KMeans(init="k-means++", n_clusters = i, n_init=30, max_iter=250, random_state=2)
    labels = clusterer.fit_predict(X_train)
    silho_score = silhouette_score(X_train, labels)
    silhouetter_coefficients.append(silho_score)
    
for i in range(len(n_silhouette)):
    print(
        "For the number of clusters = ",
        n_silhouette[i],
        "the silhouette coefficient is: ",
        silhouetter_coefficients[i])
    


"""
Perform the following steps:

1. Loop over the various possible K values you wish to test
2. Initialize a K means object.
3. Fit the training data on the K means object.
4. Use the silhouette score method available from the sklearn metrics.
5. Append the score to the silhouetter_coefficients list.
6. Display the the silhouette coefficient associated with each value of K.
"""




    

For the number of clusters =  2 the silhouette coefficient is:  0.5156651273289213
For the number of clusters =  3 the silhouette coefficient is:  0.44940874625232297
For the number of clusters =  4 the silhouette coefficient is:  0.4497013662925309
For the number of clusters =  5 the silhouette coefficient is:  0.40162831955026396
For the number of clusters =  6 the silhouette coefficient is:  0.35010480412415323
For the number of clusters =  7 the silhouette coefficient is:  0.33199362010419853
For the number of clusters =  8 the silhouette coefficient is:  0.31490974729029664
For the number of clusters =  9 the silhouette coefficient is:  0.32735720959140135
For the number of clusters =  10 the silhouette coefficient is:  0.32411928547119145


'\nPerform the following steps:\n\n1. Loop over the various possible K values you wish to test\n2. Initialize a K means object.\n3. Fit the training data on the K means object.\n4. Use the silhouette score method available from the sklearn metrics.\n5. Append the score to the silhouetter_coefficients list.\n6. Display the the silhouette coefficient associated with each value of K.\n'

# K Means

In [68]:
kmeans = KMeans(
    init="k-means++",
    n_clusters = 2, #Input the value you configured using the Silhouette coefficient analysis.
    n_init=30,
    max_iter=250,
    random_state=2
)

#TO-DO
# Fit to the training data
kmeans = kmeans.fit(X_train)
#TO-DO
# Add the features and the training data you used to the variable below.
training_df_clustered = pd.DataFrame(X_train, columns=['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR', 'CGPA', 'Research'])

#TO-DO
# Predict clusters for the training data
train_cluster = kmeans.predict(training_df_clustered)

#TO-DO
# Add the target and predicted clusters to the training DataFrame
training_df_clustered['Target cluster'] = kmeans.labels_.tolist()
training_df_clustered['cluster'] = train_cluster.tolist()


#TO-DO
# Set the number of clusters based on the silhouette coefficient analysis
number_cluster = 2

X_train_clusters_df = []
for i in range(number_cluster):
    X_train_clusters_df.append(training_df_clustered[training_df_clustered['cluster']==i])

C:\Users\garda\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


# Building Linear Regression for our clusters

In [69]:
from sklearn.linear_model import LinearRegression

"""
The number of clusters would be defined by the outcome of the silhouetter coefficient 
Set up the model of Linear Regression by exploring the different parameters: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
train_clusters_df is a dataframe that contains both the true cluster values and the predicted cluster values. Feel free to change the variable name to something else if you have been following a different naming convention.
"""

# Set the number of clusters based on the silhouette coefficient analysis
number_cluster = 2
obj_cluster = []

for i in range(number_cluster):
    model = LinearRegression()
    X_clustered_data = X_train_clusters_df[i].iloc[:,:-2]
    Y_clustered_data = X_train_clusters_df[i].iloc[:,-1]
    obj_cluster.append(model.fit(X_clustered_data, Y_clustered_data)) #Replace the underlines with the variable name you used to create the Linear Regression object.

In [70]:
def predict_value(x_test, kmeans, cluster_linear):
  """
  Input: 
  x_test is the test value that you wish to predict on.
  kmeans is the kmeans object that you have finalized to predict on the test dataset.
  cluster_linear is the list of fitted models on different clusters.

  Return:
  linear_pred - linear_pred will be type list with prediction values
  clusters - clusters_pred will be the prediction of clusters using k means.

  Follow these steps:
  1. Predict clusters using K means object on the test data.
  2. Predict regression values using Linear Regression list.
  3. return both the predictions.

  """
  linear_pred = [cluster_linear[1].predict(x_test)]
  clusters = [kmeans.predict(x_test)]
  return linear_pred, clusters

# Final Steps

In [71]:
#Apply the clustering-based linear regression to the test set.
Y_svr_k_means_pred, clusters = predict_value(X_test, kmeans, obj_cluster)

C:\Users\garda\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [72]:
print('test error (MSE):: ', get_mse(Y_test, np.array(Y_svr_k_means_pred).reshape(120,1)))
print('test error (MAE): ', get_mae(Y_test, np.array(Y_svr_k_means_pred).reshape(120,1)))

True
test error (MSE)::  0.09840083333333333
True
test error (MAE):  0.28075
